In [1]:
from cnf import *

In [2]:
from sympy.logic import simplify_logic
from sympy.logic.inference import satisfiable
from sympy.parsing.sympy_parser import parse_expr

In [3]:
import z3

In [4]:
import pycosat

In [5]:
SAMPLE = 1000
K = 3
N = 4
M = 30

In [6]:
import time

def timed(func):
    def new_func(*args, **kwargs):
        start = time.time()
        results = func(*args, **kwargs)
        end = time.time()
        print("Function '{}' took: {} s".format(func.__name__, end-start))
        return results
    return new_func

In [7]:
cnfs = timed(get_random_kcnfs)(SAMPLE, K, N, M)

Function 'get_random_kcnfs' took: 0.14688634872436523 s


In [8]:
def is_solveable(form):
    s = z3.Solver()
    s.add(form)
    r = s.check()
    if r == z3.unsat:
        return False
    elif r == z3.sat:
        return True
    else:
        raise ValueError(r)

In [9]:
# sat/unsat?
@timed
def get_sats_sympy(cnfs):
    results = []
    for cnf in cnfs:
        result = bool(satisfiable(parse_expr(str(cnf))))
        results.append(cnf.satisfiable())
    return results

@timed
def get_sats_z3(cnfs):
    results = []
    z3vars = [z3.Bool('v'+str(i)) for i in range(100)]
    for cnf in cnfs:
        form = z3.And(*[
            z3.Or(*[z3vars[abs(sv)] if sv > 0 else z3.Not(z3vars[abs(sv)])
                    for sv in clause])
            for clause in cnf.clauses])
        results.append(is_solveable(form))
    return results

@timed
def get_sats_pyco(cnfs):
    results = []
    for cnf in cnfs:
        results.append(pycosat.solve(cnf.clauses) != 'UNSAT')
    return results

In [10]:
cnfs = timed(get_random_kcnfs)(1000, 2, 3, 20)
r_sp = get_sats_sympy(cnfs)
r_z3 = get_sats_z3(cnfs)
r_pc = get_sats_pyco(cnfs)
assert r_sp == r_z3 == r_pc
print("OK")

Function 'get_random_kcnfs' took: 0.07337188720703125 s
Function 'get_sats_sympy' took: 5.878615379333496 s
Function 'get_sats_z3' took: 10.537521362304688 s
Function 'get_sats_pyco' took: 0.009574651718139648 s
OK


In [11]:
cnfs = timed(get_random_kcnfs)(100, 3, 4, 30)
r_sp = get_sats_sympy(cnfs)
r_z3 = get_sats_z3(cnfs)
r_pc = get_sats_pyco(cnfs)
assert r_sp == r_z3 == r_pc
print("OK")

Function 'get_random_kcnfs' took: 0.02122950553894043 s
Function 'get_sats_sympy' took: 1.701305627822876 s
Function 'get_sats_z3' took: 1.2968173027038574 s
Function 'get_sats_pyco' took: 0.0017707347869873047 s
OK


In [12]:
cnfs = timed(get_random_kcnfs)(100, 3, 5, 30)
r_sp = get_sats_sympy(cnfs)
r_z3 = get_sats_z3(cnfs)
r_pc = get_sats_pyco(cnfs)
assert r_sp == r_z3 == r_pc
print("OK")

Function 'get_random_kcnfs' took: 0.019964218139648438 s
Function 'get_sats_sympy' took: 2.144693613052368 s
Function 'get_sats_z3' took: 1.3028674125671387 s
Function 'get_sats_pyco' took: 0.002234220504760742 s
OK


In [13]:
cnfs = timed(get_random_kcnfs)(1000, 3, 7, 30)
r_sp = get_sats_sympy(cnfs)
r_z3 = get_sats_z3(cnfs)
r_pc = get_sats_pyco(cnfs)
assert r_sp == r_z3 == r_pc
print("OK")

Function 'get_random_kcnfs' took: 0.14324021339416504 s
Function 'get_sats_sympy' took: 24.55345368385315 s
Function 'get_sats_z3' took: 14.368957281112671 s
Function 'get_sats_pyco' took: 0.017174243927001953 s
OK


In [14]:
pycosat.solve([])

[]

In [15]:
pycosat.solve([[]])

'UNSAT'